# Webscraping : Bulk Approach

In [2]:
import time
import requests
import numpy as np
import pandas as pd

## PARAMETERS

In [3]:
URL =  """https://geoportal.nic.in/nicgis/rest/services/SCHOOLGIS/Schooldata/MapServer/0/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometry=%7B%22xmin%22%3A8922952.933909904%2C%22ymin%22%3A3052589.161589872%2C%22xmax%22%3A9001224.450873956%2C%22ymax%22%3A3130860.678553924%2C%22spatialReference%22%3A%7B%22wkid%22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100"""

In [4]:
def get_bulk_df(URL, xmin, xmax, ymin, ymax):

    new_API = URL.replace("xmin%22%3A8922952.933909904", f"xmin%22%3A{xmin}")

    new_API = new_API.replace("ymin%22%3A3052589.161589872", f"ymin%22%3A{ymin}")
    new_API = new_API.replace("xmax%22%3A9001224.450873956", f"xmax%22%3A{xmax}")
    new_API = new_API.replace("ymax%22%3A3130860.678553924", f"ymax%22%3A{ymax}")

    r = requests.get(URL)
    features = len(r.json()['features'])
    bulk_data = r.json()['features']
    bulk_df = pd.DataFrame([])
    for feature in np.arange(0, features): 
        df = pd.DataFrame(bulk_data[feature]['attributes'], index=[feature])
        bulk_df = pd.concat([bulk_df, df], ignore_index=True)

    bulk_df = bulk_df[['objectid_1', 'fid_school', 'objectid', 'schcd', 'schname', 'schcat',
        'school_cat', 'schtype', 'school_typ', 'schmgt', 'management', 'rururb', 'location', 
        'pincode', 'dtname', 'udise_stco', 'stname', 'longitude', 'latitude', 'stcode11', 'dtcode11']]

    bulk_df.rename(columns={"schcd": "school_code", "schname": "school_name"}, inplace=True)
    bulk_df.sort_values(by="objectid_1")

    return bulk_df

In [5]:
x_array_min = np.arange(63.833724747188256, 110.91562588827199, 0.1)
x_array_max = x_array_min + 1
y_array_min = np.arange(6.8114540002900235, 42.416072435295774, 0.1)
y_array_max = y_array_min + 1

In [6]:
len(x_array_min)

471

In [37]:
x_array_min[0]

63.833724747188256

### Sólo Moviendonos sobre X, dejando 'Y' fijo

In [7]:
all_dfs = pd.DataFrame([])

for x_value in x_array_max:

    for y_value in y_array_min:
        x_min = x_array_min[counter]
        y_min = y_array_min[0]
        x_max = x_array_max[counter]
        y_max = y_array_max[1]

        bulk_df = get_bulk_df(URL,x_min, x_max, y_min, y_max)

        all_dfs = pd.concat([all_dfs, bulk_df], ignore_index=True)

        time.sleep(np.random.randint(0, 2))

ConnectionError: HTTPSConnectionPool(host='geoportal.nic.in', port=443): Max retries exceeded with url: /nicgis/rest/services/SCHOOLGIS/Schooldata/MapServer/0/query?f=json&returnGeometry=true&spatialRel=esriSpatialRelIntersects&geometry=%7B%22xmin%22%3A8922952.933909904%2C%22ymin%22%3A3052589.161589872%2C%22xmax%22%3A9001224.450873956%2C%22ymax%22%3A3130860.678553924%2C%22spatialReference%22%3A%7B%22wkid%22%3A102100%7D%7D&geometryType=esriGeometryEnvelope&inSR=102100&outFields=*&outSR=102100 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000024790760370>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [8]:
all_dfs

,objectid_1,fid_school,objectid,school_code,school_name,schcat,school_cat,schtype,school_typ,schmgt,...,rururb,location,pincode,dtname,udise_stco,stname,longitude,latitude,stcode11,dtcode11
0,1094761,1355556,1370593,9340903210,VIVEKANAND INTER COLLEGE,5,Higher Secondary (6-12),3,Co-Ed,5,...,1,,208001,Kanpur Nagar,09,UTTAR PRADESH,80.161600,26.458000,09,164
1,1099336,1110519,1118565,9340302606,MAHARSHI DAYANAND VIDYAPITH,1,Primary,3,Co-Ed,5,...,1,,209307,Kanpur Nagar,09,UTTAR PRADESH,80.161438,26.476768,09,164
2,1099482,1348799,1363679,9340302605,MAA SATYAWATI JHS DOOL,4,Upper Primary Only,3,Co-Ed,5,...,1,,209307,Kanpur Nagar,09,UTTAR PRADESH,80.160018,26.477413,09,164
3,1099486,1112081,1120189,9340302607,ADARSH SHIKSHA N.,1,Primary,3,Co-Ed,5,...,1,,209307,Kanpur Nagar,09,UTTAR PRADESH,80.160063,26.477435,09,164
4,1099487,1167595,1178165,9340302604,MAA SATYAWATI P.V. DOOL,1,Primary,3,Co-Ed,5,...,1,,209307,Kanpur Nagar,09,UTTAR PRADESH,80.160072,26.477442,09,164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219995,1130596,660600,665204,9260909202,P.M.PYAREPUR,4,Upper Primary Only,3,Co-Ed,1,...,1,,209801,Unnao,09,UTTAR PRADESH,80.347805,26.641634,09,156
219996,1130600,1109212,1117189,9261102705,M.P. R.R.PUBLIC SCHOOL,1,Primary,3,Co-Ed,5,...,1,,209801,Unnao,09,UTTAR PRADESH,80.347720,26.641660,09,156
219997,1132051,75086,75933,9261104002,PS MASWASI,1,Primary,3,Co-Ed,1,...,1,,209862,Unnao,09,UTTAR PRADESH,80.348660,26.650610,09,156
219998,1137419,352858,355849,9260408901,P.S. NYAMAT PUR,1,Primary,3,Co-Ed,1,...,1,,209871,Unnao,09,UTTAR PRADESH,80.347023,26.682963,09,156
